# Data

- https://vk.com/u_samovaraa?w=wall-81871567_61842
- https://vk.com/vkys_nos?w=wall-41960737_13333
- https://vk.com/receptik_kulinar?w=wall-59496329_52708
- https://vk.com/lisimnik_cake?w=wall-82240292_25648
- https://vk.com/kingcook?w=wall-59442940_11047
- https://vk.com/u_samovaraa?w=wall-81871567_61917
- https://vk.com/quickrecipes?w=wall-61337543_5892
- https://vk.com/namenuru?w=wall-36303114_56579
- https://vk.com/vegan_cookbook?w=wall-43818640_25903
- https://vk.com/multivarka_cook?w=wall-51300483_11948

In [1]:
from glob import iglob as list_paths


def load_text(path):
    with open(path) as file:
        return file.read()


texts = sorted(
    load_text(_)
    for _ in list_paths('texts/*.txt')
)

In [2]:
def make_translation(source, target):
    assert len(source) == len(target)
    return {
        ord(a): ord(b)
        for a, b in zip(source, target)
    }


DASHES_TRANSLATION = make_translation(
    '‑–—−',
    '----'
)


def preprocess(text):
    text = text.replace('\xa0', ' ')
    text = text.replace('\xad', '')
    text = text.translate(DASHES_TRANSLATION)
    return text


texts = [preprocess(_) for _ in texts]

In [3]:
print(texts[0])

"Баранки"

Баранки хорошо подходят к чаю, кофе и молоку. У изделий хрустящая корочка и рассыпчатый мякиш. Идеальны для детей как сладкое лакомство.
Совет шеф-повара: Полейте баранки жидким шоколадом или посыпьте сахарной пудрой. Добавьте в тесто обжаренные кунжутные семечки, это придаст выпечке легкий восточный колорит.

Количество порций - 16 шт. (по 60 г)
Время приготовления - 45 мин

Ингредиенты:

450 г пшеничной муки
2 яйца
50 мл растительного масла
200 мл молока + 20 мл на смазывание
10 ч. л. сахара
1 ч. л. соды
1 ч. л. соли
15 г муки на подпыл

Еще больше рецептов в приложении для iPad и iPhone "Готовят все" https://itunes.apple.com/ru/app/id587314984


In [4]:
import re


def find_ingredient_sections(text):
    return re.findall(r'ингредиенты:(.+?)приготовление', text, re.I | re.S)



def maybe_ingredient(line):
    match = re.search(r'\d', line)
    size = len(line) <= 50
    return match and size


lines = []
for text in texts:
    sections = find_ingredient_sections(text)
    for section in sections:
        for line in section.splitlines():
            if maybe_ingredient(line):
                lines.append(line)

In [5]:
from random import seed, sample

seed(1)
sample(lines, 20)

['Майонез - 2 ст. л. ',
 'Яйцо - 1 шт.',
 '1 ст. л. уксуса',
 '● сахар - 1/2 стакана;',
 ' Сахар - 3 ст.ложки,',
 '100 г мякоти тыквы ',
 'Огурец - 1 шт. ',
 '85 гр зелени',
 '- 1 стакан пшеничной муки',
 ' Черная смородина (у меня замороженная) - 1 ст.',
 'морковь - 1 кг ',
 '1/2 ч.ложки сахара',
 '● 1 ч.л. сухих дрожжей (или 12 г свежих)',
 '3-4 зубчика чеснока ',
 '- Огурец свежий 2 шт.',
 '120 мл оливкового масла плюс 2 ст. ложки',
 'Яйцо - 3 шт ',
 '● сметана - 200 г;',
 '- мука 2 стакана ',
 '* 3/4 стакана холодной кипяченой воды']

In [6]:
len(lines)

4054

# Grammar

In [7]:
from yargy.tokenizer import MorphTokenizer

TOKENIZER = MorphTokenizer()
list(TOKENIZER('рулетики'))

[MorphToken(
     value='рулетики',
     span=[0, 8),
     type='RU',
     forms=[Form('рулетик', Grams(NOUN,inan,masc,nomn,plur)),
      Form('рулетик', Grams(NOUN,accs,inan,masc,plur)),
      Form('рулетик', Grams(NOUN,inan,masc,nomn,plur)),
      Form('рулетик', Grams(NOUN,accs,inan,masc,plur)),
      Form('рулётик', Grams(NOUN,inan,masc,nomn,plur)),
      Form('рулётик', Grams(NOUN,accs,inan,masc,plur))]
 )]

In [8]:
# %run -n main.py
# parser = Parser(INGREDIENT)
# seed(1)
# for line in sample(lines, 100):
#     matches = list(parser.findall(line))
#     spans = [_.span for _ in matches]
#     show_markup(line, spans)
# #     if matches:
# #         match = matches[0]
# #         display(match.tree.as_dot)
# #         display(match.fact)